In [21]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

pd.set_option("display.max_colwidth", 80)

In [6]:
baseurl = "https://medley.life"
testurl = "https://medley.life/institutions/pref_"

# user agent
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
}

In [25]:
sum = 0
for pref in prefs:
    res = requests.get(testurl+pref, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    sum += int(re.search(r"\d+", soup.find("span", "fs-m-s").get_text().strip()).group())

In [26]:
sum

177709

In [4]:
address= [test.find("a").get("href") for test in tests]

In [5]:
# 詳細ページ
print(address[0])
# res = requests.get(baseurl+address[0], headers=headers)
res = requests.get(testurl, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")


/institutions/564ab1321c1b604d59c177da/


# 探索ループ

In [3]:
# ドメイン
base_url = "https://medley.life"
# target_url = "/institutions/pref_北海道/?page="

# user agent
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
}

prefs = ['北海道', '青森県', '岩手県', '宮城県', '秋田県', 
             '山形県', '福島県', '茨城県', '栃木県', '群馬県', 
             '埼玉県', '千葉県', '東京都', '神奈川県', '新潟県', 
             '富山県', '石川県', '福井県', '山梨県', '長野県', 
             '岐阜県', '静岡県', '愛知県', '三重県', '滋賀県', 
             '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県', 
             '鳥取県', '島根県', '岡山県', '広島県', '山口県', 
             '徳島県', '香川県', '愛媛県', '高知県', '福岡県', 
             '佐賀県', '長崎県', '熊本県', '大分県', '宮崎県', 
             '鹿児島県', '沖縄県']

In [27]:
prefs[::-1]

['沖縄県',
 '鹿児島県',
 '宮崎県',
 '大分県',
 '熊本県',
 '長崎県',
 '佐賀県',
 '福岡県',
 '高知県',
 '愛媛県',
 '香川県',
 '徳島県',
 '山口県',
 '広島県',
 '岡山県',
 '島根県',
 '鳥取県',
 '和歌山県',
 '奈良県',
 '兵庫県',
 '大阪府',
 '京都府',
 '滋賀県',
 '三重県',
 '愛知県',
 '静岡県',
 '岐阜県',
 '長野県',
 '山梨県',
 '福井県',
 '石川県',
 '富山県',
 '新潟県',
 '神奈川県',
 '東京都',
 '千葉県',
 '埼玉県',
 '群馬県',
 '栃木県',
 '茨城県',
 '福島県',
 '山形県',
 '秋田県',
 '宮城県',
 '岩手県',
 '青森県',
 '北海道']

In [7]:
for pref in prefs:
    target_url = "/institutions/pref_"+pref+"/?page="
    # 初期化
    df_all = pd.DataFrame()
    page = 1
    last_flag = False

    while True:
        url = base_url + target_url + str(page)
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")

        if soup.find(class_="o-pagination__next") is not None or last_flag is True:
            # ページ表示
            print("--- Getting data at page = {} ---".format(page))
            time.sleep(3) # 遅延させる

            hospitals = soup.find_all("li", "c-search-item")

            # hospital内探索
            for hospital in tqdm(hospitals):
                toadd = {}

                hospital_url = hospital.find("a").get("href")
                hospital_type = hospital.find("label").get_text().strip()
                toadd["タイプ"] = hospital_type
                # time.sleep(1)
                # 病院ページにアクセス
                res = requests.get(base_url+hospital_url, headers=headers)
                soup = BeautifulSoup(res.text, "html.parser")

                rows = soup.find_all("table")[0].find_all("tr")

                for row in rows:
                    toadd[row.th.get_text()] = row.td.get_text()

                # 診療科と専門外来を追加
                for section in soup.find_all("section"):
                    if "診療科" in section.find("header").get_text().strip():
                        departments = section.find_all("a")
                        tmp = [department.get_text().strip() for department in departments]
                        toadd["診療科"] = " ".join(tmp)

                    if "専門外来" in section.find("header").get_text().strip():
                        outpatients = section.find_all("label")
                        tmp = [outpatient.get_text().strip() for outpatient in outpatients]
                        toadd["専門外来"] = " ".join(tmp)

                # データフレームに格納
                df = pd.io.json.json_normalize(toadd)
                try:
                    df_all = pd.concat([df_all, df])
                except NameError:
                    df_all = pd.io.json.json_normalize(toadd)

            page += 1

            if last_flag:
                df_all.to_csv("./"+pref+".csv")
                print("--- DONE. ---")
                break
        else:
            if last_flag is True:
                break
            else:
                last_flag = True

--- Getting data at page = 281 ---


100%|██████████| 25/25 [00:05<00:00,  4.44it/s]


--- Getting data at page = 282 ---


100%|██████████| 15/15 [00:03<00:00,  4.05it/s]

--- DONE. ---


In [183]:
# \nが気持ち悪い場合はmap
remove_n = lambda x: x.lstrip()
df_all["施設名"] = df_all["施設名"].map(remove_n)

In [9]:
df_all.to_csv("medley_test.csv")

In [28]:
pd.read_csv("/Users/YukiOta/Desktop/nicoly_2700.csv")

,Unnamed: 0,SNS,こだわり条件,ブログ,メディカルローン,交通手段,休業日,住所,備考,公式サイト,医院名,対応クレジットカード,支払い方法,施設情報,診療時間,電話番号,駐車場
0,0,NaN,NaN,NaN,クリニックにお問い合わせください,地下鉄天神駅、西鉄福岡駅より徒歩3分,水曜・木曜,〒810-0001 福岡県福岡市中央区天神1-10-13 天神MMTビル9階,NaN,http://tenjin-skin.com/,天神あおばクリニック,NaN,NaN,NaN,10:30～12:30 14:00～18:30,092-791-3111,記載なし
1,0,NaN,NaN,NaN,クリニックにお問い合わせください,地下鉄鶴見緑地線「ドーム前千代崎駅」「大正駅」から徒歩8分 JR大阪環状線「大正駅」から徒歩8分 阪神なんば線「ドーム前駅」「九条駅」から徒歩8分,土曜午後・日曜・祝日,〒550-0024 大阪府大阪市西区境川1丁目1番31号 境川メディカルセンター3F,NaN,http://www.sakurakomachi-clinic.com/,さくらこまち皮フ科クリニック,NaN,NaN,NaN,月・木：9:30〜12:30 14:30〜17:30 火：9:30〜18:30 水・金：9:30〜12:30 14:30〜18:30 土：9:30〜12:30,06-6583-1112,提携駐車場あり
2,0,NaN,NaN,NaN,クリニックにお問い合わせください,日比谷線 東銀座駅 徒歩1分,祝日,〒104-0061 東京都中央区銀座5丁目14番15号 サンリット銀座ビル9F,NaN,http://www.ginza-hakusan.com/,銀座白山クリニック,NaN,NaN,NaN,11:00～20:00（月〜日）,03-6228-4911,記載なし
3,0,NaN,NaN,NaN,クリニックにお問い合わせください,地下鉄堺筋・谷町線「天神橋筋六丁目駅」11番出口から徒歩約5分,水曜午後・土曜午後・日曜・祝日,〒531-0074 大阪府大阪市北区本庄東1-24-8,NaN,http://www.kyoiin.com/,京医院,NaN,NaN,NaN,■ 外来 9:00〜12:00 16:30〜18:30 ■ リハビリ室 9:00〜12:00 14:00〜18:30,06-6371-0080,専用駐車場あり
4,0,NaN,NaN,NaN,クリニックにお問い合わせください,JR大阪駅前、ヒルトンホテル斜め向側,水曜日・日曜日,〒530-0001 大阪府大阪市北区梅田2-1-22 野村不動産西梅田ビル8F,NaN,http://www.fujiiclinic-umeda.com/,藤井クリニック,NaN,NaN,NaN,10:00〜19:00（祝日も診察）,0120-482-212,近隣駐車場あり
5,0,NaN,NaN,NaN,クリニックにお問い合わせください,JR蕨（わらび）駅 東口方面 バス停「イオンモール川口キャラ」から徒歩1分,火曜午後・土曜午後・日曜日,〒333-0842 埼玉県川口市前川1-1-55 メディパーク川口前川1F,NaN,http://www.m-skinclinic.com/,前川皮フ科・形成外科クリニック,NaN,NaN,NaN,9:00〜18:30,048-264-1733,記載なし
6,0,NaN,NaN,NaN,クリニックにお問い合わせください,JR武蔵野線「東川口」駅,水曜日・土曜日の午後・日曜日・祝日,〒333-0811 埼玉県川口市戸塚1-5-30,NaN,http://www.megumi-hifuka.com/,めぐみ皮フ科クリニック,NaN,NaN,NaN,9:30〜13:00 15:00〜17:30,048-291-5266,無料駐車場 10台
7,0,NaN,NaN,NaN,クリニックにお問い合わせください,JR藤枝駅 南口2階より直結,火曜日・水曜日,〒426-0067 静岡県藤枝市前島1-3-1 ホテルオーレ藤枝 3階,NaN,https://www.skinclinic-fujieda.com/,スキンクリニック藤枝,NaN,NaN,NaN,10:00〜19:00,054-631-9806,あり
8,0,NaN,NaN,NaN,クリニックにお問い合わせください,秋田駅から徒歩約7分,水曜日・日曜日,〒010-0001 秋田県秋田市中通一丁目4-3 エリアなかいち@4の31階,NaN,http://clinic.adeb.com/,AdeBクリニック秋田本院,NaN,NaN,NaN,10:00〜13:00 14:00〜18:00,018-836-1112,なかいち駐車場（30分まで無料、1時間につき100円）
9,0,NaN,NaN,NaN,クリニックにお問い合わせください,名鉄名古屋本線 豊明駅1番出口から徒歩約2分,土曜午後・日曜・祝日,〒470-1144 愛知県豊明市阿野町西ノ海戸16-1,NaN,http://risei-kai.or.jp/hp/,前原外科・整形外科 小児科,NaN,NaN,NaN,8:30〜12:00 17:30〜20:00,0562-92-1170,記載なし
